In [6]:
!uv add textblob

Resolved 104 packages in 278ms                                       
Prepared 1 package in 205ms                                              
Installed 1 package in 3ms                                  
 + textblob==0.19.0


In [7]:
"""
Twitter Archive Consciousness Analyzer
Processes Twitter/X archives to identify potential navigators and their consciousness patterns

This script analyzes tweets to detect:
1. Complexity battles (insurance, healthcare, taxes, etc.)
2. Consciousness capacity (ability to maintain coherent awareness)
3. Pattern recognition abilities
4. Care drivers and formative experiences
5. Time Violence incidents
"""

import json
import re
import hashlib
from datetime import datetime, timedelta
from typing import Dict, List, Tuple, Optional, Set
from dataclasses import dataclass, field
from collections import defaultdict, Counter
import sqlite3
import nltk
from textblob import TextBlob
import numpy as np

In [8]:

# Download required NLTK data
try:
    nltk.download('punkt', quiet=True)
    nltk.download('stopwords', quiet=True)
    nltk.download('vader_lexicon', quiet=True)
except:
    pass

from nltk.sentiment import SentimentIntensityAnalyzer

# Initialize sentiment analyzer
sia = SentimentIntensityAnalyzer()

@dataclass
class ConsciousnessIndicator:
    """Represents a moment of consciousness demonstration in tweets"""
    tweet_id: str
    timestamp: datetime
    content: str
    complexity_domain: str
    consciousness_type: str  # 'battle', 'pattern', 'solution', 'liberation'
    intensity_score: float
    situation_hash: str
    
@dataclass
class NavigatorProfile:
    """Potential navigator profile built from Twitter data"""
    twitter_handle: str
    email: Optional[str] = None
    
    # Consciousness capacity indicators
    complexity_battles: List[Dict] = field(default_factory=list)
    conscious_domains: Dict[str, Dict] = field(default_factory=dict)
    pattern_recognitions: List[Dict] = field(default_factory=list)
    
    # Metrics
    fragmentation_resistance: int = 5
    consciousness_sharing_ability: int = 5
    pattern_recognition_depth: int = 5
    
    # Essential characteristics
    care_drivers: Set[str] = field(default_factory=set)
    formative_experiences: List[str] = field(default_factory=list)
    complexity_scale: str = 'component'
    consciousness_stage: str = 'emerging'
    
    # Time Violence tracking
    time_violence_incidents: List[Dict] = field(default_factory=list)
    total_hours_lost: float = 0

In [19]:



class TwitterConsciousnessAnalyzer:
    """Analyzes Twitter archives for consciousness patterns"""

    def _initialize_database(self):
        """Initialize SQLite database with Bottega schema"""
        conn = sqlite3.connect(self.db_path)
        cur = conn.cursor()
        
        # Create tables if they don't exist
        cur.executescript("""
            CREATE TABLE IF NOT EXISTS navigators (
                navigator_id TEXT PRIMARY KEY DEFAULT (lower(hex(randomblob(16)))),
                email TEXT UNIQUE NOT NULL,
                phone TEXT,
                experiences TEXT NOT NULL, -- JSON
                max_hours_per_week INTEGER DEFAULT 35,
                unavailable_topics TEXT DEFAULT '[]', -- JSON array
                support_group_id TEXT,
                created_at TIMESTAMP DEFAULT CURRENT_TIMESTAMP,
                
                CHECK (max_hours_per_week <= 40)
            );
            
            CREATE TABLE IF NOT EXISTS consciousness_capacity (
                navigator_id TEXT PRIMARY KEY REFERENCES navigators(navigator_id),
                complexity_scale TEXT NOT NULL CHECK (
                    complexity_scale IN ('component', 'subsystem', 'system', 'ecosystem')
                ),
                conscious_domains TEXT NOT NULL, -- JSON
                fragmentation_resistance INTEGER NOT NULL CHECK (fragmentation_resistance BETWEEN 1 AND 10),
                consciousness_sharing_ability INTEGER NOT NULL CHECK (consciousness_sharing_ability BETWEEN 1 AND 10),
                pattern_recognition_depth INTEGER NOT NULL CHECK (pattern_recognition_depth BETWEEN 1 AND 10),
                care_drivers TEXT NOT NULL, -- JSON array
                formative_experiences TEXT NOT NULL, -- JSON array
                consciousness_stage TEXT DEFAULT 'emerging' CHECK (
                    consciousness_stage IN ('emerging', 'developing', 'integrated', 'mastery')
                )
            );
            
            CREATE TABLE IF NOT EXISTS time_violence_incidents (
                incident_id TEXT PRIMARY KEY DEFAULT (lower(hex(randomblob(16)))),
                navigator_id TEXT REFERENCES navigators(navigator_id),
                tweet_id TEXT,
                timestamp TEXT,
                domain TEXT,
                description TEXT,
                estimated_hours REAL,
                created_at TIMESTAMP DEFAULT CURRENT_TIMESTAMP
            );
            
            CREATE TABLE IF NOT EXISTS consciousness_indicators (
                indicator_id TEXT PRIMARY KEY DEFAULT (lower(hex(randomblob(16)))),
                navigator_id TEXT REFERENCES navigators(navigator_id),
                tweet_id TEXT,
                timestamp TEXT,
                complexity_domain TEXT,
                consciousness_type TEXT,
                intensity_score REAL,
                situation_hash TEXT,
                content TEXT,
                created_at TIMESTAMP DEFAULT CURRENT_TIMESTAMP
            );
        """)
        
        conn.commit()
        conn.close()

    def get_database_statistics(self) -> Dict:
        """Get statistics from the database"""
        conn = sqlite3.connect(self.db_path)
        cur = conn.cursor()
        
        stats = {}
        
        # Total navigators
        cur.execute("SELECT COUNT(*) FROM navigators")
        stats['total_navigators'] = cur.fetchone()[0]
        
        # Consciousness distribution
        cur.execute("""
            SELECT consciousness_stage, COUNT(*) 
            FROM consciousness_capacity 
            GROUP BY consciousness_stage
        """)
        stats['consciousness_stages'] = dict(cur.fetchall())
        
        # Total Time Violence
        cur.execute("SELECT SUM(estimated_hours) FROM time_violence_incidents")
        result = cur.fetchone()[0]
        stats['total_time_violence_hours'] = result if result else 0
        
        # Top domains
        cur.execute("""
            SELECT domain, COUNT(*), SUM(estimated_hours)
            FROM time_violence_incidents
            GROUP BY domain
            ORDER BY SUM(estimated_hours) DESC
            LIMIT 5
        """)
        stats['top_time_violence_domains'] = [
            {'domain': row[0], 'incidents': row[1], 'hours': row[2]}
            for row in cur.fetchall()
        ]
        
        conn.close()
        return stats    
    

        
    # Complexity domain keywords
    DOMAIN_PATTERNS = {
        'healthcare': {
            'keywords': ['insurance', 'claim', 'denied', 'appeal', 'prior auth', 
                        'coverage', 'deductible', 'copay', 'formulary', 'medical bill',
                        'ADHD', 'medication', 'pharmacy', 'doctor', 'specialist'],
            'time_violence_indicators': ['on hold', 'hours', 'waiting', 'finally', 
                                        'months', 'fighting', 'exhausted'],
            'consciousness_indicators': ['realized', 'pattern', 'always', 'every time',
                                        'figured out', 'trick is', 'learned']
        },
        'financial': {
            'keywords': ['debt', 'bankruptcy', 'collections', 'credit', 'loan',
                        'student loans', 'interest', 'payment plan', 'foreclosure',
                        'eviction', 'overdraft', 'bank'],
            'time_violence_indicators': ['paperwork', 'documents', 'forms', 'calls',
                                        'letters', 'notices', 'deadlines'],
            'consciousness_indicators': ['systemic', 'rigged', 'designed to', 'trap',
                                        'escape', 'beat the system', 'loophole']
        },
        'employment': {
            'keywords': ['job search', 'interview', 'resume', 'ATS', 'application',
                        'rejection', 'ghosted', 'salary', 'negotiation', 'linkedin',
                        'unemployed', 'laid off', 'fired'],
            'time_violence_indicators': ['applications', 'rounds', 'months looking',
                                        'never heard back', 'automated rejection'],
            'consciousness_indicators': ['game', 'algorithm', 'keyword', 'hack',
                                        'actually works', 'real secret']
        },
        'government': {
            'keywords': ['DMV', 'IRS', 'tax', 'benefits', 'disability', 'SSA',
                        'SNAP', 'welfare', 'unemployment', 'permits', 'license'],
            'time_violence_indicators': ['queue', 'line', 'website down', 'office',
                                        'appointment', 'weeks wait', 'processing'],
            'consciousness_indicators': ['bureaucracy', 'kafkaesque', 'catch-22',
                                        'workaround', 'backdoor', 'right person']
        },
        'education': {
            'keywords': ['FAFSA', 'financial aid', 'tuition', 'transcript', 'credits',
                        'registration', 'advisor', 'requirements', 'prerequisite',
                        'degree', 'graduation'],
            'time_violence_indicators': ['runaround', 'different answers', 'conflicting',
                                        'no one knows', 'sent between departments'],
            'consciousness_indicators': ['navigate', 'system works', 'hidden requirement',
                                        'unwritten rule', 'actually need']
        }
    }
    
    # Neurodivergent indicators (often correlated with consciousness workers)
    NEURODIVERGENT_PATTERNS = [
        'ADHD', 'autism', 'autistic', 'AuDHD', 'neurodivergent', 'ND',
        'executive dysfunction', 'rejection sensitive', 'RSD', 'hyperfocus',
        'stimming', 'masking', 'burnout', 'meltdown', 'shutdown',
        'sensory', 'overwhelm', 'spoons', 'dysregulation'
    ]
    
    def __init__(self, db_path: str = "bottega.db"):
        """Initialize analyzer with SQLite database path"""
        self.db_path = db_path
        self.navigator_profile = None
        self.consciousness_indicators = []
        self._initialize_database()
        
    def load_twitter_archive(self, archive_path: str) -> List[Dict]:
        """Load tweets from Twitter archive file"""
        tweets = []
        
        # Twitter archives usually have tweets in data/tweets.js or similar
        try:
            with open(archive_path, 'r', encoding='utf-8') as f:
                content = f.read()
                
                # Remove JavaScript variable assignment if present
                if content.startswith('window.YTD.tweets.part0 ='):
                    content = content.replace('window.YTD.tweets.part0 =', '')
                elif content.startswith('window.YTD.tweet.part0 ='):
                    content = content.replace('window.YTD.tweet.part0 =', '')
                    
                data = json.loads(content)
                
                # Extract tweet objects (structure varies)
                for item in data:
                    if 'tweet' in item:
                        tweets.append(item['tweet'])
                    else:
                        tweets.append(item)
                        
        except Exception as e:
            print(f"Error loading archive: {e}")
            
        return tweets
    
    def analyze_consciousness_patterns(self, tweets: List[Dict]) -> NavigatorProfile:
        """Analyze tweets for consciousness patterns"""
        
        # Extract basic info from first tweet
        twitter_handle = tweets[0].get('user', {}).get('screen_name', 'unknown')
        profile = NavigatorProfile(twitter_handle=twitter_handle)
        
        # Process tweets chronologically
        sorted_tweets = sorted(tweets, key=lambda x: x.get('created_at', ''))
        
        for tweet in sorted_tweets:
            self._analyze_single_tweet(tweet, profile)
        
        # Calculate consciousness metrics based on patterns found
        self._calculate_consciousness_metrics(profile)
        
        # Determine consciousness scale and stage
        self._determine_consciousness_level(profile)
        
        return profile
    
    def _analyze_single_tweet(self, tweet: Dict, profile: NavigatorProfile):
        """Analyze individual tweet for consciousness indicators"""
        
        text = tweet.get('full_text', tweet.get('text', '')).lower()
        created_at = datetime.strptime(tweet['created_at'], '%a %b %d %H:%M:%S %z %Y')
        tweet_id = tweet.get('id_str', '')
        
        # Check each complexity domain
        for domain, patterns in self.DOMAIN_PATTERNS.items():
            
            # Count keyword matches
            keyword_matches = sum(1 for kw in patterns['keywords'] if kw in text)
            
            if keyword_matches > 0:
                
                # Check for Time Violence
                tv_indicators = sum(1 for ind in patterns['time_violence_indicators'] if ind in text)
                if tv_indicators > 0:
                    time_wasted = self._extract_time_amount(text)
                    profile.time_violence_incidents.append({
                        'tweet_id': tweet_id,
                        'timestamp': created_at.isoformat(),
                        'domain': domain,
                        'description': text[:200],
                        'estimated_hours': time_wasted
                    })
                    profile.total_hours_lost += time_wasted
                
                # Check for consciousness indicators
                consciousness_matches = sum(1 for ind in patterns['consciousness_indicators'] if ind in text)
                if consciousness_matches > 0:
                    
                    # This shows pattern recognition / consciousness development
                    situation_hash = hashlib.md5(f"{domain}:{text[:100]}".encode()).hexdigest()[:16]
                    
                    indicator = ConsciousnessIndicator(
                        tweet_id=tweet_id,
                        timestamp=created_at,
                        content=text,
                        complexity_domain=domain,
                        consciousness_type=self._classify_consciousness_type(text),
                        intensity_score=self._calculate_intensity(text),
                        situation_hash=situation_hash
                    )
                    
                    self.consciousness_indicators.append(indicator)
                    
                    # Track domain consciousness
                    if domain not in profile.conscious_domains:
                        profile.conscious_domains[domain] = {}
                    
                    profile.conscious_domains[domain][situation_hash] = 'developing'
                    
                    # Add to formative experiences if intense enough
                    if indicator.intensity_score > 0.7:
                        profile.formative_experiences.append(
                            f"{domain}_battle_{created_at.year}"
                        )
                
                # Track care drivers
                sentiment = sia.polarity_scores(text)
                if sentiment['compound'] < -0.5 or sentiment['compound'] > 0.5:
                    profile.care_drivers.add(f"{domain}_reform")
        
        # Check for neurodivergent patterns
        nd_matches = sum(1 for pattern in self.NEURODIVERGENT_PATTERNS if pattern in text)
        if nd_matches > 0:
            profile.care_drivers.add('neurodivergent_advocacy')
    
    def _extract_time_amount(self, text: str) -> float:
        """Extract time amounts from text"""
        hours = 0.0
        
        # Look for explicit hour mentions
        hour_patterns = [
            r'(\d+)\s*hours?',
            r'(\d+)\s*hrs?',
            r'(\d+)\s*minutes?',
            r'(\d+)\s*mins?',
            r'(\d+)\s*days?',
            r'(\d+)\s*weeks?',
            r'(\d+)\s*months?'
        ]
        
        for pattern in hour_patterns:
            matches = re.findall(pattern, text)
            for match in matches:
                amount = float(match)
                if 'min' in pattern:
                    hours += amount / 60
                elif 'day' in pattern:
                    hours += amount * 8  # Assume 8 hours per day of dealing with complexity
                elif 'week' in pattern:
                    hours += amount * 20  # Assume 20 hours per week
                elif 'month' in pattern:
                    hours += amount * 80  # Assume 80 hours per month
                else:
                    hours += amount
        
        # If no explicit time, estimate based on keywords
        if hours == 0:
            if any(word in text for word in ['all day', 'entire day', 'whole day']):
                hours = 8
            elif any(word in text for word in ['all morning', 'all afternoon']):
                hours = 4
            elif any(word in text for word in ['finally', 'eventually', 'at last']):
                hours = 2  # Implied significant time
        
        return hours
    
    def _classify_consciousness_type(self, text: str) -> str:
        """Classify the type of consciousness demonstration"""
        
        if any(word in text for word in ['fighting', 'battle', 'struggle', 'dealing']):
            return 'battle'
        elif any(word in text for word in ['realized', 'pattern', 'always', 'every']):
            return 'pattern'
        elif any(word in text for word in ['solution', 'fixed', 'solved', 'hack']):
            return 'solution'
        elif any(word in text for word in ['free', 'escaped', 'done', 'over']):
            return 'liberation'
        else:
            return 'awareness'
    
    def _calculate_intensity(self, text: str) -> float:
        """Calculate intensity of consciousness demonstration"""
        
        sentiment = sia.polarity_scores(text)
        
        # Strong emotions (positive or negative) indicate intensity
        emotional_intensity = abs(sentiment['compound'])
        
        # Profanity often indicates frustration with systems
        profanity_indicators = ['fuck', 'shit', 'damn', 'hell', 'bullshit', 'wtf']
        profanity_score = sum(0.1 for word in profanity_indicators if word in text.lower())
        
        # Capitalization indicates emphasis
        caps_ratio = sum(1 for c in text if c.isupper()) / max(len(text), 1)
        
        # Exclamation marks indicate intensity
        exclamation_score = text.count('!') * 0.1
        
        intensity = min(1.0, emotional_intensity + profanity_score + caps_ratio + exclamation_score)
        
        return intensity
    
    def _calculate_consciousness_metrics(self, profile: NavigatorProfile):
        """Calculate consciousness metrics from patterns"""
        
        # Fragmentation resistance - based on number of battles survived
        battles_survived = len(profile.complexity_battles) + len(profile.time_violence_incidents)
        if battles_survived > 20:
            profile.fragmentation_resistance = 9
        elif battles_survived > 10:
            profile.fragmentation_resistance = 7
        elif battles_survived > 5:
            profile.fragmentation_resistance = 6
        
        # Pattern recognition depth - based on patterns identified
        patterns_found = len(profile.pattern_recognitions)
        for domain_patterns in profile.conscious_domains.values():
            patterns_found += len(domain_patterns)
        
        if patterns_found > 15:
            profile.pattern_recognition_depth = 9
        elif patterns_found > 8:
            profile.pattern_recognition_depth = 7
        elif patterns_found > 3:
            profile.pattern_recognition_depth = 6
        
        # Consciousness sharing ability - based on helping others
        # Look for replies, quote tweets with advice, threads explaining systems
        # (This would need more sophisticated analysis of tweet interactions)
        # For now, estimate based on consciousness demonstrations
        
        if len(self.consciousness_indicators) > 20:
            profile.consciousness_sharing_ability = 8
        elif len(self.consciousness_indicators) > 10:
            profile.consciousness_sharing_ability = 6
    
    def _determine_consciousness_level(self, profile: NavigatorProfile):
        """Determine consciousness scale and stage"""
        
        # Determine scale based on breadth and depth
        domains_engaged = len(profile.conscious_domains)
        total_patterns = sum(len(p) for p in profile.conscious_domains.values())
        
        if domains_engaged >= 3 and total_patterns > 20:
            profile.complexity_scale = 'system'
        elif domains_engaged >= 2 and total_patterns > 10:
            profile.complexity_scale = 'subsystem'
        else:
            profile.complexity_scale = 'component'
        
        # Determine stage based on metrics
        avg_metric = (profile.fragmentation_resistance + 
                     profile.consciousness_sharing_ability + 
                     profile.pattern_recognition_depth) / 3
        
        if avg_metric >= 8:
            profile.consciousness_stage = 'mastery'
        elif avg_metric >= 7:
            profile.consciousness_stage = 'integrated'
        elif avg_metric >= 5:
            profile.consciousness_stage = 'developing'
        else:
            profile.consciousness_stage = 'emerging'
    
    def save_to_database(self, profile: NavigatorProfile):
        """Save navigator profile to SQLite database"""
        
        conn = sqlite3.connect(self.db_path)
        cur = conn.cursor()
        
        try:
            # Generate navigator ID
            import uuid
            navigator_id = str(uuid.uuid4())
            
            # Insert navigator
            cur.execute("""
                INSERT INTO navigators (navigator_id, email, phone, experiences, max_hours_per_week, 
                                      unavailable_topics, support_group_id)
                VALUES (?, ?, ?, ?, ?, ?, ?)
            """, (
                navigator_id,
                profile.email or f"{profile.twitter_handle}@twitter.import",
                None,  # No phone from Twitter
                json.dumps({
                    'source': 'twitter',
                    'domains': list(profile.conscious_domains.keys()),
                    'battles': profile.formative_experiences
                }),
                35,  # Default max hours
                json.dumps([]),  # No unavailable topics yet
                None  # Assign support group later
            ))
            
            # Insert consciousness capacity
            cur.execute("""
                INSERT INTO consciousness_capacity 
                (navigator_id, complexity_scale, conscious_domains, fragmentation_resistance,
                 consciousness_sharing_ability, pattern_recognition_depth, care_drivers,
                 formative_experiences, consciousness_stage)
                VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?)
            """, (
                navigator_id,
                profile.complexity_scale,
                json.dumps(profile.conscious_domains),
                profile.fragmentation_resistance,
                profile.consciousness_sharing_ability,
                profile.pattern_recognition_depth,
                json.dumps(list(profile.care_drivers)),
                json.dumps(profile.formative_experiences),
                profile.consciousness_stage
            ))
            
            # Insert Time Violence incidents
            for incident in profile.time_violence_incidents:
                cur.execute("""
                    INSERT INTO time_violence_incidents
                    (navigator_id, tweet_id, timestamp, domain, description, estimated_hours)
                    VALUES (?, ?, ?, ?, ?, ?)
                """, (
                    navigator_id,
                    incident['tweet_id'],
                    incident['timestamp'],
                    incident['domain'],
                    incident['description'],
                    incident['estimated_hours']
                ))
                print(f"Time Violence found: {incident['domain']} - {incident['estimated_hours']} hours lost")
            
            # Insert consciousness indicators
            for indicator in self.consciousness_indicators:
                cur.execute("""
                    INSERT INTO consciousness_indicators
                    (navigator_id, tweet_id, timestamp, complexity_domain, consciousness_type,
                     intensity_score, situation_hash, content)
                    VALUES (?, ?, ?, ?, ?, ?, ?, ?)
                """, (
                    navigator_id,
                    indicator.tweet_id,
                    indicator.timestamp.isoformat(),
                    indicator.complexity_domain,
                    indicator.consciousness_type,
                    indicator.intensity_score,
                    indicator.situation_hash,
                    indicator.content[:500]  # Limit content length
                ))
            
            conn.commit()
            print(f"Navigator profile saved with ID: {navigator_id}")
            print(f"Total Time Violence documented: {profile.total_hours_lost} hours")
            print(f"Consciousness indicators saved: {len(self.consciousness_indicators)}")
            
            return navigator_id
            
        except Exception as e:
            conn.rollback()
            print(f"Error saving to database: {e}")
            import traceback
            traceback.print_exc()
            return None
        finally:
            conn.close()
    
    def generate_report(self, profile: NavigatorProfile) -> str:
        """Generate human-readable report of findings"""
        
        report = f"""
===========================================
CONSCIOUSNESS WORKER ANALYSIS REPORT
Twitter Handle: @{profile.twitter_handle}
===========================================

CONSCIOUSNESS CAPACITY ASSESSMENT:
----------------------------------
Complexity Scale: {profile.complexity_scale}
Consciousness Stage: {profile.consciousness_stage}

Metrics (1-10 scale):
- Fragmentation Resistance: {profile.fragmentation_resistance}
- Consciousness Sharing Ability: {profile.consciousness_sharing_ability}
- Pattern Recognition Depth: {profile.pattern_recognition_depth}

CONSCIOUS DOMAINS:
-----------------
"""
        
        for domain, patterns in profile.conscious_domains.items():
            report += f"\n{domain.upper()}:\n"
            report += f"  - Patterns recognized: {len(patterns)}\n"
        
        report += f"""
FORMATIVE EXPERIENCES:
---------------------
"""
        for exp in profile.formative_experiences[:5]:  # Top 5
            report += f"- {exp}\n"
        
        report += f"""
CARE DRIVERS:
------------
"""
        for driver in profile.care_drivers:
            report += f"- {driver}\n"
        
        report += f"""
TIME VIOLENCE SUMMARY:
---------------------
Total incidents documented: {len(profile.time_violence_incidents)}
Total hours lost to complexity: {profile.total_hours_lost:.1f}
Estimated annual Time Violence: {profile.total_hours_lost * 12:.0f} hours

Top Time Violence Domains:
"""
        
        domain_hours = defaultdict(float)
        for incident in profile.time_violence_incidents:
            domain_hours[incident['domain']] += incident['estimated_hours']
        
        for domain, hours in sorted(domain_hours.items(), key=lambda x: x[1], reverse=True)[:3]:
            report += f"  - {domain}: {hours:.1f} hours\n"
        
        report += """
NAVIGATOR POTENTIAL:
-------------------
"""
        
        if profile.consciousness_stage in ['integrated', 'mastery']:
            report += "✓ STRONG CANDIDATE - High consciousness capacity demonstrated\n"
        elif profile.consciousness_stage == 'developing':
            report += "✓ GOOD CANDIDATE - Developing consciousness capacity\n"
        else:
            report += "○ EMERGING - Early stage consciousness development\n"
        
        report += f"""
This person has demonstrated the ability to maintain conscious awareness
of complex systems that have caused them harm. Their lived experience
has value and could help others navigate similar complexity.

RECOMMENDED VENTURES:
"""
        
        for domain in list(profile.conscious_domains.keys())[:3]:
            report += f"  - {domain.capitalize()} Navigation\n"
        
        report += """
===========================================
"""
        
        return report

def main():
    """Main execution function"""
    
    # Database path (SQLite)
    db_path = "bottega.db"
    
    # Initialize analyzer
    analyzer = TwitterConsciousnessAnalyzer(db_path)
    
    # Load Twitter archive
    archive_path = input("Enter path to Twitter archive file (tweets.js) or press Enter for demo: ").strip()
    
    if not archive_path:
        # Use example data for testing
        print("Using example tweets for demonstration...")
        tweets = [
            {
                'id_str': '1234567890',
                'created_at': 'Mon Oct 10 15:30:45 +0000 2023',
                'full_text': 'Day 47 of fighting insurance for ADHD meds. 6 hours on hold today. Finally got through only to be told I need a different form. The pattern is always the same - they exhaust you into giving up.',
                'user': {'screen_name': 'complexity_survivor'}
            },
            {
                'id_str': '1234567891',
                'created_at': 'Tue Oct 11 09:15:30 +0000 2023',
                'full_text': 'REALIZED: Every insurance denial uses the same three tactics. Document everything, appeal immediately, mention state insurance commissioner. Works every time. The system is designed to break you.',
                'user': {'screen_name': 'complexity_survivor'}
            },
            {
                'id_str': '1234567892',
                'created_at': 'Wed Oct 12 14:22:18 +0000 2023',
                'full_text': 'Thread: How I finally beat the prior auth system after 3 years of fighting 🧵\n\n1/ The key is understanding they profit from friction...',
                'user': {'screen_name': 'complexity_survivor'}
            },
            {
                'id_str': '1234567893',
                'created_at': 'Thu Oct 13 11:45:00 +0000 2023',
                'full_text': 'Applied to 200+ jobs this month. ATS systems are filtering out qualified candidates. The trick? Mirror exact keywords from job posting. Broken system but here we are.',
                'user': {'screen_name': 'complexity_survivor'}
            },
            {
                'id_str': '1234567894',
                'created_at': 'Fri Oct 14 16:20:15 +0000 2023',
                'full_text': 'ADHD tax is real. Spent another 4 hours trying to remember passwords and find documents for taxes. Why does every system assume perfect executive function?',
                'user': {'screen_name': 'complexity_survivor'}
            }
        ]
    else:
        tweets = analyzer.load_twitter_archive(archive_path)
    
    if not tweets:
        print("No tweets found in archive")
        return
    
    print(f"\nLoaded {len(tweets)} tweets for analysis...")
    print("="*50)
    
    # Analyze consciousness patterns
    profile = analyzer.analyze_consciousness_patterns(tweets)
    
    # Generate report
    report = analyzer.generate_report(profile)
    print(report)
    
    # Always save to database (SQLite is local)
    print("\nSaving to local SQLite database...")
    navigator_id = analyzer.save_to_database(profile)
    
    if navigator_id:
        print(f"✓ Navigator profile successfully created: {navigator_id}")
        print(f"✓ Database location: {db_path}")
        
        # Show database statistics
        print("\n" + "="*50)
        print("DATABASE STATISTICS:")
        print("-"*50)
        stats = analyzer.get_database_statistics()
        print(f"Total Navigators: {stats['total_navigators']}")
        print(f"Total Time Violence Documented: {stats['total_time_violence_hours']:.1f} hours")
        
        if stats['consciousness_stages']:
            print("\nConsciousness Stage Distribution:")
            for stage, count in stats['consciousness_stages'].items():
                print(f"  {stage}: {count}")
        
        if stats['top_time_violence_domains']:
            print("\nTop Time Violence Domains:")
            for domain_stat in stats['top_time_violence_domains']:
                print(f"  {domain_stat['domain']}: {domain_stat['hours']:.1f} hours ({domain_stat['incidents']} incidents)")
    
    # Export consciousness indicators for further analysis
    export_indicators = input("\nExport consciousness indicators to JSON? (y/n): ").strip().lower()
    if export_indicators == 'y':
        output_file = f"consciousness_indicators_{profile.twitter_handle}.json"
        with open(output_file, 'w') as f:
            json.dump(
                {
                    'navigator_id': navigator_id,
                    'twitter_handle': profile.twitter_handle,
                    'analysis_date': datetime.now().isoformat(),
                    'profile_summary': {
                        'consciousness_scale': profile.complexity_scale,
                        'consciousness_stage': profile.consciousness_stage,
                        'total_time_violence_hours': profile.total_hours_lost,
                        'domains': list(profile.conscious_domains.keys()),
                        'care_drivers': list(profile.care_drivers)
                    },
                    'consciousness_indicators': [
                        {
                            'tweet_id': ind.tweet_id,
                            'timestamp': ind.timestamp.isoformat(),
                            'domain': ind.complexity_domain,
                            'type': ind.consciousness_type,
                            'intensity': ind.intensity_score,
                            'content': ind.content[:200]
                        }
                        for ind in analyzer.consciousness_indicators
                    ]
                },
                f,
                indent=2
            )
        print(f"Consciousness indicators exported to {output_file}")
    
    print("\n" + "="*50)
    print("ANALYSIS COMPLETE")
    print(f"Navigator potential identified. Welcome to the Bottega network.")
    print("Your trauma is your qualification. Your consciousness is your contribution.")
    print("="*50)



In [20]:
main()

Enter path to Twitter archive file (tweets.js) or press Enter for demo:  /Users/leoguinan/Downloads/Manual Library/twitter-2025-09-12-665d8dcc3c02ac164b910fdf4474d6e19296464a57b75b7039df8330f83a6b99/data/tweets.js



Loaded 35390 tweets for analysis...

CONSCIOUSNESS WORKER ANALYSIS REPORT
Twitter Handle: @unknown

CONSCIOUSNESS CAPACITY ASSESSMENT:
----------------------------------
Complexity Scale: system
Consciousness Stage: mastery

Metrics (1-10 scale):
- Fragmentation Resistance: 9
- Consciousness Sharing Ability: 8
- Pattern Recognition Depth: 9

CONSCIOUS DOMAINS:
-----------------

HEALTHCARE:
  - Patterns recognized: 18

EMPLOYMENT:
  - Patterns recognized: 10

FINANCIAL:
  - Patterns recognized: 8

FORMATIVE EXPERIENCES:
---------------------
- healthcare_battle_2025
- healthcare_battle_2025
- employment_battle_2023
- financial_battle_2025
- healthcare_battle_2025

CARE DRIVERS:
------------
- financial_reform
- neurodivergent_advocacy
- education_reform
- employment_reform
- healthcare_reform
- government_reform

TIME VIOLENCE SUMMARY:
---------------------
Total incidents documented: 71
Total hours lost to complexity: 2168.2
Estimated annual Time Violence: 26018 hours

Top Time Viole


Export consciousness indicators to JSON? (y/n):  y


Consciousness indicators exported to consciousness_indicators_unknown.json

ANALYSIS COMPLETE
Navigator potential identified. Welcome to the Bottega network.
Your trauma is your qualification. Your consciousness is your contribution.
